In [1]:
import numpy as np
import pandas as pd

from load import read_csv, to_csv

In [2]:
path = './Data/bt-impute.csv'

df = read_csv(path)
print(len(df))
df.head()

53096


,TUMOURID,PATIENTID,DIAGNOSISDATEBEST,SITE_ICD10_O2_3CHAR,MORPH_ICD10_O2,BEHAVIOUR_ICD10_O2,STAGE_BEST,GRADE,AGE,SEX,...,SACT,CLINICAL_TRIAL,CHEMO_RADIATION,REGIMEN_COUNT,CLINICAL_TRIAL_INDICATOR,CHEMO_RADIATION_INDICATOR,NORMALISED_HEIGHT,NORMALISED_WEIGHT,DAYS_TO_FIRST_SURGERY,DAYS_SINCE_DIAGNOSIS
0,10614165,10501771,2017-01-16,C71,9440.0,3,U,G4,69.0,1,...,False,False,False,0.0,False,False,0.000,0.00,0,751
1,10614166,10501772,2015-11-10,C71,9440.0,3,U,G4,88.0,1,...,True,False,False,1.0,False,False,1.755,71.75,0,17
2,10614166,10501772,2015-11-10,C71,9440.0,3,U,G4,88.0,1,...,True,False,False,3.0,False,False,1.755,71.75,0,17
3,10614166,10501772,2015-11-10,C71,9440.0,3,U,G4,88.0,1,...,True,False,False,2.0,False,False,1.755,71.75,0,17
4,10614166,10501772,2015-11-10,C71,9440.0,3,U,G4,88.0,1,...,True,False,False,4.0,False,False,1.755,71.75,0,17


In [3]:
df = df.drop(df[(df.SITE_ICD10_O2_3CHAR != 'C70') & 
                    (df.SITE_ICD10_O2_3CHAR != 'C71') & 
                    (df.SITE_ICD10_O2_3CHAR != 'C72') & 
                    (df.SITE_ICD10_O2_3CHAR != 'D32') &
                    (df.SITE_ICD10_O2_3CHAR != 'D33') &
                    (df.SITE_ICD10_O2_3CHAR != 'D35')].index)
df.SITE_ICD10_O2_3CHAR.value_counts()

C71    31478
D32    11606
D33     4622
D35     4145
C72      844
C70      306
C13        0
D41        0
D09        0
D37        0
D43        0
C91        0
D47        0
C44        0
C51        0
C54        0
C67        0
C77        0
C84        0
C92        0
D01        0
C80        0
C82        0
C17        0
C20        0
C21        0
C22        0
C23        0
C24        0
C25        0
C43        0
C48        0
C50        0
C53        0
C64        0
C65        0
C73        0
C78        0
C79        0
D03        0
Name: SITE_ICD10_O2_3CHAR, dtype: int64

In [4]:
df = df.drop(columns = ['TUMOURID',	'PATIENTID'	,'DIAGNOSISDATEBEST','LINKNUMBER'])

In [5]:
site_dummies = pd.get_dummies(df.SITE_ICD10_O2_3CHAR, prefix='SITE')
df = df.drop(columns = 'SITE_ICD10_O2_3CHAR')
df = pd.concat([df, site_dummies], axis=1)
df.head()


,MORPH_ICD10_O2,BEHAVIOUR_ICD10_O2,STAGE_BEST,GRADE,AGE,SEX,CREG_CODE,LATERALITY,QUINTILE_2015,TUMOUR_COUNT,...,SITE_D43,SITE_D47,SITE_C44,SITE_C51,SITE_C54,SITE_C67,SITE_C77,SITE_C84,SITE_C92,SITE_D03
0,9440.0,3,U,G4,69.0,1,L0401,R,4,2,...,0,0,0,0,0,0,0,0,0,0
1,9440.0,3,U,G4,88.0,1,L1001,R,1,2,...,0,0,0,0,0,0,0,0,0,0
2,9440.0,3,U,G4,88.0,1,L1001,R,1,2,...,0,0,0,0,0,0,0,0,0,0
3,9440.0,3,U,G4,88.0,1,L1001,R,1,2,...,0,0,0,0,0,0,0,0,0,0
4,9440.0,3,U,G4,88.0,1,L1001,R,1,2,...,0,0,0,0,0,0,0,0,0,0


In [6]:
df.MORPH_ICD10_O2.value_counts().to_csv('./Data/morph.csv', index=True)
# too little variations
df = df.drop(columns='MORPH_ICD10_O2')

In [7]:
df.BEHAVIOUR_ICD10_O2 = df.BEHAVIOUR_ICD10_O2.replace({2: 0, 5: 3, 6: 3, 9: 3})
df = df.drop(df[df.BEHAVIOUR_ICD10_O2 == 1].index)
df.BEHAVIOUR_ICD10_O2.value_counts()

# note that pytorch requires float32 instead of int
# transform elsewhere
df['BENIGN_BEHAVIOUR'] = (df.BEHAVIOUR_ICD10_O2 == 0).astype(int)
df = df.drop(columns= 'BEHAVIOUR_ICD10_O2')
df.head()

,STAGE_BEST,GRADE,AGE,SEX,CREG_CODE,LATERALITY,QUINTILE_2015,TUMOUR_COUNT,ETHNICITY,NEWVITALSTATUS,...,SITE_D47,SITE_C44,SITE_C51,SITE_C54,SITE_C67,SITE_C77,SITE_C84,SITE_C92,SITE_D03,BENIGN_BEHAVIOUR
0,U,G4,69.0,1,L0401,R,4,2,W,A,...,0,0,0,0,0,0,0,0,0,0
1,U,G4,88.0,1,L1001,R,1,2,W,D,...,0,0,0,0,0,0,0,0,0,0
2,U,G4,88.0,1,L1001,R,1,2,W,D,...,0,0,0,0,0,0,0,0,0,0
3,U,G4,88.0,1,L1001,R,1,2,W,D,...,0,0,0,0,0,0,0,0,0,0
4,U,G4,88.0,1,L1001,R,1,2,W,D,...,0,0,0,0,0,0,0,0,0,0


In [8]:
df = df.drop(columns='STAGE_BEST')
df.head()

,GRADE,AGE,SEX,CREG_CODE,LATERALITY,QUINTILE_2015,TUMOUR_COUNT,ETHNICITY,NEWVITALSTATUS,SACT,...,SITE_D47,SITE_C44,SITE_C51,SITE_C54,SITE_C67,SITE_C77,SITE_C84,SITE_C92,SITE_D03,BENIGN_BEHAVIOUR
0,G4,69.0,1,L0401,R,4,2,W,A,False,...,0,0,0,0,0,0,0,0,0,0
1,G4,88.0,1,L1001,R,1,2,W,D,True,...,0,0,0,0,0,0,0,0,0,0
2,G4,88.0,1,L1001,R,1,2,W,D,True,...,0,0,0,0,0,0,0,0,0,0
3,G4,88.0,1,L1001,R,1,2,W,D,True,...,0,0,0,0,0,0,0,0,0,0
4,G4,88.0,1,L1001,R,1,2,W,D,True,...,0,0,0,0,0,0,0,0,0,0


In [9]:
grade_mapper = {'GX' : -1, 'G0' : 0, 'G1' : 1, 'G2' : 2, 'G3' : 3, 'G4' : 4}
df.GRADE = df.GRADE.replace(grade_mapper)
df.head()

,GRADE,AGE,SEX,CREG_CODE,LATERALITY,QUINTILE_2015,TUMOUR_COUNT,ETHNICITY,NEWVITALSTATUS,SACT,...,SITE_D47,SITE_C44,SITE_C51,SITE_C54,SITE_C67,SITE_C77,SITE_C84,SITE_C92,SITE_D03,BENIGN_BEHAVIOUR
0,4,69.0,1,L0401,R,4,2,W,A,False,...,0,0,0,0,0,0,0,0,0,0
1,4,88.0,1,L1001,R,1,2,W,D,True,...,0,0,0,0,0,0,0,0,0,0
2,4,88.0,1,L1001,R,1,2,W,D,True,...,0,0,0,0,0,0,0,0,0,0
3,4,88.0,1,L1001,R,1,2,W,D,True,...,0,0,0,0,0,0,0,0,0,0
4,4,88.0,1,L1001,R,1,2,W,D,True,...,0,0,0,0,0,0,0,0,0,0


In [10]:
creg_dummies = pd.get_dummies(df.CREG_CODE, prefix='CREG')
df = df.drop(columns = 'CREG_CODE')
df = pd.concat([df, creg_dummies], axis=1)
df.head()

,GRADE,AGE,SEX,LATERALITY,QUINTILE_2015,TUMOUR_COUNT,ETHNICITY,NEWVITALSTATUS,SACT,CLINICAL_TRIAL,...,SITE_D03,BENIGN_BEHAVIOUR,CREG_L0201,CREG_L0301,CREG_L0401,CREG_L0801,CREG_L0901,CREG_L1001,CREG_L1201,CREG_L1701
0,4,69.0,1,R,4,2,W,A,False,False,...,0,0,0,0,1,0,0,0,0,0
1,4,88.0,1,R,1,2,W,D,True,False,...,0,0,0,0,0,0,0,1,0,0
2,4,88.0,1,R,1,2,W,D,True,False,...,0,0,0,0,0,0,0,1,0,0
3,4,88.0,1,R,1,2,W,D,True,False,...,0,0,0,0,0,0,0,1,0,0
4,4,88.0,1,R,1,2,W,D,True,False,...,0,0,0,0,0,0,0,1,0,0


In [11]:
lat_dummies = pd.get_dummies(df.LATERALITY, prefix='LAT')
df = df.drop(columns = 'LATERALITY')
df = pd.concat([df, lat_dummies], axis=1)

In [12]:
eth_dummies = pd.get_dummies(df.ETHNICITY, prefix='ETH')
df = df.drop(columns = 'ETHNICITY')
df = pd.concat([df, eth_dummies], axis=1)

In [13]:
# only 80 censored
# EVENT means not alive 
df = df.drop(df[df.NEWVITALSTATUS == 'X'].index)
vital_mapper = {'A' : 0, 'D' : 1}
df['EVENT'] = df.NEWVITALSTATUS.replace(vital_mapper)
df = df.drop(columns='NEWVITALSTATUS')


In [14]:
df.SACT = df.SACT.astype(int)
df.CLINICAL_TRIAL_INDICATOR = df.CLINICAL_TRIAL_INDICATOR.astype(int)
df.CHEMO_RADIATION_INDICATOR = df.CHEMO_RADIATION_INDICATOR.astype(int)

In [15]:
df.head()

,GRADE,AGE,SEX,QUINTILE_2015,TUMOUR_COUNT,SACT,CLINICAL_TRIAL,CHEMO_RADIATION,REGIMEN_COUNT,CLINICAL_TRIAL_INDICATOR,...,LAT_M,LAT_R,ETH_A,ETH_B,ETH_C,ETH_M,ETH_O,ETH_U,ETH_W,EVENT
0,4,69.0,1,4,2,0,False,False,0.0,0,...,0,1,0,0,0,0,0,0,1,0
1,4,88.0,1,1,2,1,False,False,1.0,0,...,0,1,0,0,0,0,0,0,1,1
2,4,88.0,1,1,2,1,False,False,3.0,0,...,0,1,0,0,0,0,0,0,1,1
3,4,88.0,1,1,2,1,False,False,2.0,0,...,0,1,0,0,0,0,0,0,1,1
4,4,88.0,1,1,2,1,False,False,4.0,0,...,0,1,0,0,0,0,0,0,1,1


In [16]:
# unscaled pandas 
to_csv(df, './Data/data.csv', index=False)

# this would give headerless numpy 
# data = scale(df)
# np.savetxt("./Data/data.csv", data, delimiter=",")

In [17]:
pd.set_option('display.max_rows', None)
df.dtypes


GRADE                           int64
AGE                           float64
SEX                          category
QUINTILE_2015                  object
TUMOUR_COUNT                    int64
SACT                            int64
CLINICAL_TRIAL                   bool
CHEMO_RADIATION                  bool
REGIMEN_COUNT                 float64
CLINICAL_TRIAL_INDICATOR        int64
CHEMO_RADIATION_INDICATOR       int64
NORMALISED_HEIGHT             float64
NORMALISED_WEIGHT             float64
DAYS_TO_FIRST_SURGERY           int64
DAYS_SINCE_DIAGNOSIS            int64
SITE_C13                        uint8
SITE_C17                        uint8
SITE_C20                        uint8
SITE_C21                        uint8
SITE_C22                        uint8
SITE_C23                        uint8
SITE_C24                        uint8
SITE_C25                        uint8
SITE_C43                        uint8
SITE_C48                        uint8
SITE_C50                        uint8
SITE_C53    